In [1]:
import sys

sys.path.append('E:/UADY/CARLA/CARLA_Latest/WindowsNoEditor/PythonAPI/carla')

import carla
import numpy as np
import re
import cv2
import time

In [11]:
class Simulation_parking:
    def __init__(self):
        self.client = carla.Client('localhost', 2000)
        self.world = self.client.get_world()
        self.vehicle_all = self.world.get_blueprint_library().filter('*vehicle*')
        self.car_tesla = self.world.get_blueprint_library().filter('*model3*')
        self.exclude_regex = re.compile(
            r'.*(cybertruck|ambulance|fusorosa|vespa|kawasaki|crossbike|harley|micro|carlamotors|yamaha|gazelle|diamondback).*')
        self.vehicle_all = [bp for bp in self.world.get_blueprint_library().filter('*vehicle*') if
                            not self.exclude_regex.match(bp.id)]
        self.available_parking_probability = 0.33
        self.spectator_line = 1
        self.parking_points = []
        self.is_parkable_from = []
        self.lines_of_parking_x = [10, 3.5, -7.0, -13.0, -23, -29]
        self.lines_of_parking_y = [-18.8, -21.6, -24.4, -27.2, -30.0, -32.8, -35.6, -38.4, -41.2, -44.0]

    def load_world(self, world_name):
        self.client.set_timeout(4.0)
        self.client.load_world(world_name)

    """
    Args:
        precipitation: float Rain intensity values range from 0 to 100
        precipitation_deposits: float Determines the creation of puddles. Values range from 0 to 100
        wind_intensity: float Controls the strenght of the wind with values from 0, no wind at all, to 100, a strong wind. The wind does affect rain direction and leaves from trees
        fog_density : float Controls the density of the fog with values from 0, no fog at all, to 100, a very dense fog
        dust_storm : float Controls the density of the dust storm with values from 0, no dust at all, to 100, a very dense dust storm
    """

    def configure_weather(self, weather):
        self.world.set_weather(weather)

    def is_available_parking(self, prob_false):
        return np.random.rand() > prob_false  #probabily of parking

    def generate_parking_points(self):
        for i, x in enumerate(self.lines_of_parking_x):
            if i == self.spectator_line:
                # print(f'{i} spectator line')
                continue
            for y in self.lines_of_parking_y:
                if self.is_available_parking(self.available_parking_probability):

                    location = carla.Transform(
                        carla.Location(x=x, y=y, z=1),
                        carla.Rotation(yaw=int(0))
                    )
                    if self.try_if_veichle_is_parkable(location):
                        self.parking_points.append(location)
                else:
                    interest_index = self.spectator_line - 1 if self.spectator_line % 2 == 0 else self.spectator_line + 1
                    if i != interest_index:
                        continue

                    x_parkable = self.lines_of_parking_x[self.spectator_line]

                    y_index = self.lines_of_parking_y.index(y)

                    point1 = [x_parkable, y]
                    if point1 not in self.is_parkable_from:
                        self.is_parkable_from.append(point1)

                    if y_index + 1 < len(self.lines_of_parking_y):
                        point2 = [x_parkable, self.lines_of_parking_y[y_index + 1]]
                        if point2 not in self.is_parkable_from:
                            self.is_parkable_from.append(point2)

                    if y_index - 1 >= 0:
                        point3 = [x_parkable, self.lines_of_parking_y[y_index - 1]]
                        if point3 not in self.is_parkable_from:
                            self.is_parkable_from.append(point3)

    def try_if_veichle_is_parkable(self, location):
        vehicle_test = self.world.try_spawn_actor(self.car_tesla[0], location)
        is_parkable = vehicle_test is not None

        if is_parkable:
            vehicle_test.destroy()

        return is_parkable

    def init_parking_vehicles(self):
        self.generate_parking_points()
        for point in self.parking_points:
            ramdom_vehicle = np.random.choice(self.vehicle_all)
            self.world.try_spawn_actor(ramdom_vehicle, point)

    def capture_image(self, image, image_path):
        image_array = np.frombuffer(image.raw_data, dtype=np.uint8)
        image_array = np.reshape(image_array, (image.height, image.width, 4))
        image_bgr = image_array[:, :, :3]  # Convertir de formato RGBA a formato BGR para OpenCV
        # Guardar la imagen en la ruta especificada
        cv2.imwrite(image_path, image_bgr)

    def make_images_from_vehicle(self, vehicle: carla.Actor, tags: str):
        ancho_imagen = 1920
        alto_imagen = 1080
        image_path = './parking_images/'
        cv2.waitKey(200)

        # Obtener la cámara frontal del vehículo
        camera_blueprint = self.world.get_blueprint_library().find('sensor.camera.rgb')
        camera_blueprint.set_attribute('image_size_x', str(ancho_imagen))
        camera_blueprint.set_attribute('image_size_y', str(alto_imagen))
        camera_blueprint.set_attribute('sensor_tick', '0.1')  # Frecuencia de actualización de la cámara en segundos

        # Configurar y spawnear la cámara frontal (hood)
        camera_location_hood = carla.Location(x=2.5, y=0.0,
                                              z=0.5)  # Ajustar la ubicación de la cámara según sea necesario
        camera_transform_hood = carla.Transform(camera_location_hood)
        camera_hood = self.world.spawn_actor(camera_blueprint, camera_transform_hood, attach_to=vehicle)

        # Configurar y spawnear la cámara frontal del espejo (mirror)
        camera_location_mirror = carla.Location(x=1, y=0.0,
                                                z=1.25)  # Ajustar la ubicación de la cámara según sea necesario
        camera_transform_mirror = carla.Transform(camera_location_mirror)
        camera_mirror = self.world.spawn_actor(camera_blueprint, camera_transform_mirror, attach_to=vehicle)

        camera_hood_tags = self.generate_tag_for_image(f'camera_front_hood_{tags}', vehicle)
        # Conectar la función de devolución de llamada a la cámara frontal (hood)
        camera_hood.listen(
            lambda image: self.capture_image(image, f'{image_path}/{camera_hood_tags}.jpg'))

        camera_mirror_tags = self.generate_tag_for_image(f'camera_front_mirror_{tags}', vehicle)
        camera_mirror.listen(
            lambda image: self.capture_image(image, f'{image_path}/{camera_mirror_tags}.jpg'))

        cv2.waitKey(110)
        camera_hood.destroy()
        camera_mirror.destroy()
        vehicle.destroy()

    def make_image_from_spectator_line(self, available_angle):
        for y in self.lines_of_parking_y:
            randon_yaw = int(np.random.choice(available_angle))
            spectator_line_value_x = self.lines_of_parking_x[self.spectator_line]
            # print(spectator_line_value_x, y)
            check_parking_point = carla.Transform(
                carla.Location(x=spectator_line_value_x, y=y, z=0.3),
                carla.Rotation(yaw=randon_yaw)
            )
            vehicle_tesla = self.world.try_spawn_actor(self.car_tesla[0], check_parking_point)
        
            if vehicle_tesla is not None:
                tag_p = True if [spectator_line_value_x, y] in self.is_parkable_from else False
                
                lights = carla.VehicleLightState(carla.VehicleLightState.Position | carla.VehicleLightState.LowBeam)
                vehicle_tesla.set_light_state(lights)
                
                self.make_images_from_vehicle(vehicle_tesla, f'p={tag_p}')

    def clean_actors(self):
        for actor in self.world.get_actors().filter('*vehicle*'):
            actor.destroy()
        for sensor in self.world.get_actors().filter('*sensor*'):
            sensor.destroy()

    def generate_tag_for_image(self, sensor_tag, vehicle):
        x = vehicle.get_location().x
        y = vehicle.get_location().y
        yaw = vehicle.get_transform().rotation.yaw
        created = time.time()
        return f's={sensor_tag}_x={x}_y={y}_a={yaw}_t={created}'

In [3]:
simulation_parking = Simulation_parking()
simulation_parking.load_world('Town05')

In [12]:
for i in [1, 2, 3, 4]:
    simulation_parking = Simulation_parking()
    simulation_parking.available_parking_probability = .9
    simulation_parking.spectator_line = i
    simulation_parking.init_parking_vehicles()
    available_angle = range(-20, 20) if i % 2 == 0 else range(160, 200)
    simulation_parking.make_image_from_spectator_line(available_angle)
    simulation_parking.clean_actors()

In [65]:
simulation_parking.clean_actors()


In [57]:
def generate_random_weathers(count_weathers):
    weathers = []
    for i in range(count_weathers):
        weathers.append(
            carla.WeatherParameters(
                # determina el angulo del sol. Valores de 0 a 100.
                sun_azimuth_angle=np.random.randint(0, 360),

                # determina la altura del sol. Valores de -90 a 90.
                sun_altitude_angle=np.random.randint(-90, 90),

                # determina la intensidad de la lluvia. Valores de 0 a 100.
                precipitation=np.random.randint(0, 50),

                # determina la creación de charcos en el suelo. Valores de 0 a 100.
                precipitation_deposits=np.random.randint(0, 50),

                # determina la intensidad del viento. Valores de 0 a 100. El viento afecta la dirección de la lluvia y las hojas de los árboles
                wind_intensity=np.random.randint(0, 50),

                # determina la densidad de la niebla. Valores de 0 a 100.
                fog_density=np.random.randint(0, 50),

                # determina la densidad de la tormenta de polvo. Valores de 0 a 100.
                dust_storm=np.random.randint(0, 50),
            )
        )
    return weathers

In [64]:
weathers = generate_random_weathers(10)
for weather in weathers:
    for i in [1, 2, 3, 4]:
        simulation_parking = Simulation_parking()
        simulation_parking.configure_weather(weather)
        simulation_parking.available_parking_probability = .4
        simulation_parking.spectator_line = i
        simulation_parking.init_parking_vehicles()
        available_angle = range(-30, 30) if i % 2 == 0 else range(150, 210)
        simulation_parking.make_image_from_spectator_line(available_angle)
        simulation_parking.clean_actors()

KeyboardInterrupt: 

In [14]:
def get_calibration_matrix(image_width , image_height, fov=90):
    focal_length = image_width / (2 * np.tan(fov * np.pi / 360))
    calibration_matrix = np.array([[focal_length, 0, image_width / 2],
                                   [0, focal_length, image_height / 2],
                                   [0, 0, 1]])
    return calibration_matrix

In [15]:
get_calibration_matrix(1920, 1080, 90)

array([[960.,   0., 960.],
       [  0., 960., 540.],
       [  0.,   0.,   1.]])